In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [2]:
from exp import nb_databunch
from exp import nb_resnet_ssd
from exp import nb_init_model

In [3]:
from exp import nb_anchors_loss_metrics

In [4]:
from exp import nb_optimizer

In [5]:
# export
import torch

In [6]:
from torch import tensor

In [7]:
from fastai.basic_train import Learner

In [8]:
from fastai.core import defaults

In [9]:
from functools import partial

In [10]:
# export
from IPython.core import debugger as idb

In [11]:
# export
import os
import shutil

In [12]:
# export
from fastai.basic_train import LearnerCallback

In [13]:
# export
from torch.utils.tensorboard import SummaryWriter

In [14]:
# export
from fastai.basic_data import DatasetType

In [15]:
# export
from collections.abc import Iterable

# functions

## get_learn

In [16]:
def get_learn(device=torch.device('cuda'),cache_img=False,opt=nb_optimizer.Adam):
    data = nb_databunch.get_databunch(data_root='./data/tiny_ds_20200331',bs=64,device=device,cache=cache_img)  
    model = nb_resnet_ssd.get_resnet18_ssd()
    model.load_state_dict(torch.load('./models/resnet18_ssd_init.pth'));
    learn = Learner(data,model)
    if device.type=='cuda':
        learn.model = torch.nn.DataParallel(learn.model,device_ids=[0,1,2,3])

    # 设置loss function
    gvs,_,_,avs,_,_ = nb_anchors_loss_metrics.get_ga()
    gaf = nb_anchors_loss_metrics.GridAnchor_Funcs(gvs,avs,device)
    learn.loss_func = partial(nb_anchors_loss_metrics.yolo_L, gaf=gaf)
    learn.metrics = [partial(nb_anchors_loss_metrics.clas_acc,gaf=gaf),
                     partial(nb_anchors_loss_metrics.cent_d,gaf=gaf),
                     partial(nb_anchors_loss_metrics.hw_r,gaf=gaf)]

    # 设置 optimizer
    learn.opt_func = opt

    return learn

## TensorboardCallback

In [17]:
# export
class TensorBoardCallback(LearnerCallback):
    def __init__(self, 
                 learn, 
                 log_dir,
                 plot_net=False,
                 plot_loss=True,
                 metric_plots=[],
                 hyper_plots=[],
                 hist_plots=[],
                 plot_paramHist=True,
                 plot_gradHist=True,
                 plot_optStateHist=True,
                 hist_iters=10e99 # 这里设置一个很大的数，防止在默认的情况下频繁地绘制hitogram拖慢训练
                ):
        '''
        参数：
        -- learn：一个Learner对象
        -- log_dir：tensorboard writer的写目录
        -- plot_net：True/False，是否绘制网络结构图，一般来说你只在第一次训练的时候绘制网络结构图
        -- plot_loss：是否绘制loss曲线，若为True，则会绘制train loss和smoothed train loss（周期为iteration）
        以及valid loss（周期为epoch）曲线
        -- metric_plots：一个list，用于指定一个或多个metric名称（字符串），被指定了metric会被绘制（周期为epoch）
        -- hyper_plots：一个list，用于指定一个或多个hyper parameter名称（字符串），被指定了的hyper parameter会被绘制（周期为iteration），
        每个param_group的该超参数都会被绘制。
        -- hist_plots：一个list，用于指定一个或多个网络参数的名称（字符串），被指定了的网络参数的直方图（如果plot_paramHist为True）、
        梯度的直方图（如果plot_gradHist为True）、其在optimizer中的各state的直方图（如果plot_optStateHist为True）会被绘制（周期由hist_iters指定）
        -- hist_iters：一个正整数，指定paramHist和gradHist的绘制的周期，单位是iteration
        '''
        self.learn = learn
        self.log_dir = log_dir
        
        self.plot_net = plot_net
        
        self.plot_loss = plot_loss
        
        self.metric_plots = metric_plots # to to: 添加检查
        self.hyper_plots = hyper_plots # to do: 添加检查
        
        self.hist_plots = hist_plots # to do: 添加检查
        self.plot_paramHist = plot_paramHist
        self.plot_gradHist = plot_gradHist
        self.plot_optStateHist = plot_optStateHist
        self.hist_iters = hist_iters
        
        # 提取learner的model，方便类内方法访问
        self.model = self.learn.model.module if isinstance(self.learn.model,torch.nn.DataParallel) else self.learn.model
        # 提取模型的所有参数的名称，方便类内方法访问
        self.param_names = [name for (name, _) in self.model.named_parameters()]

        
    def _clear_logdir(self):
        '清空summarywriter的写目录'
        if os.path.exists(self.log_dir): shutil.rmtree(self.log_dir)
        os.mkdir(self.log_dir)
        
    def _plot_netGraph(self):
        '绘制网络结构'
        x = self.learn.data.one_batch(DatasetType.Single)[0].to(self.learn.data.device)
        self.writer.add_graph(self.model, x)
    
    def _plot_trainLoss(self,iteration,**kwargs):
        self.writer.add_scalars('loss',{'train': kwargs['last_loss'],'train_smooth':kwargs['smooth_loss']},iteration)
        
    def _plot_validLoss(self,iteration,**kwargs):
        self.writer.add_scalars('loss',{'valid': kwargs['last_metrics'][0]},iteration)
            
    def _plot_metrics(self,iteration,**kwargs):
        for mn in self.metric_plots: # mn: metric name
            idx = self.learn.recorder.metrics_names.index(mn)+1
            self.writer.add_scalar(f'metrics/{mn}',kwargs['last_metrics'][idx],iteration)
            
    def _plot_hypers(self,iteration):
        for hn in self.hyper_plots: # hn: hyper-parameter name
            for i,pg in enumerate(self.learn.opt.opt.param_groups):
                hp = pg[hn] # hp: hyper-parameter
                if not isinstance(hp,Iterable):
                    self.writer.add_scalar(f'pg{i}/{hn}',hp,iteration)
                else:
                    for j,hpp in enumerate(hp):
                        self.writer.add_scalar(f'pg{i}/{hn}[{j}]',hpp,iteration)
                        
    def _plot_paramHist(self, iteration):
        if len(self.hist_plots)>0 and self.plot_paramHist:
            params = [values.clone().detach().cpu() for (_, values) in self.model.named_parameters()]
            for param_name in self.hist_plots:
                idx = self.param_names.index(param_name)
                param = params[idx]
                self.writer.add_histogram(f'paramHists/{param_name}',param, iteration)
                
    def _plot_gradHist(self, iteration):
        if len(self.hist_plots)>0 and self.plot_gradHist:
            grads = [param.grad.clone().detach().cpu() for (_, param) in self.model.named_parameters() if param.grad is not None]
            for param_name in self.hist_plots:
                idx = self.param_names.index(param_name)
                grad = grads[idx]
                self.writer.add_histogram(f'gradHists/{param_name}',grad, iteration)
                
    def _plot_optStateHist(self, iteration):
        if len(self.hist_plots)>0 and self.plot_optStateHist:
            params = [values for (_, values) in self.model.named_parameters()]
            for param_name in self.hist_plots:
                idx = self.param_names.index(param_name)
                param = params[idx]
                state = self.learn.opt.opt.state[param]
                for k,v in state.items():
                    if k!='step':
                        self.writer.add_histogram(f'optStateHists/{param_name}/{k}',v, iteration, bins='fd')
                
                
    def on_train_begin(self, **kwargs):
        self._clear_logdir()
        self.writer = SummaryWriter(log_dir=self.log_dir)
        if self.plot_net: self._plot_netGraph() 

            
    def on_backward_begin(self,iteration, **kwargs):
        if self.plot_loss: self._plot_trainLoss(iteration,**kwargs)
            
            
    def on_backward_end(self, iteration, **kwargs):
        if iteration%self.hist_iters: return
        self._plot_gradHist(iteration)
    
    
    def on_step_end(self, iteration, **kwargs):
        self._plot_hypers(iteration)
        if (iteration+1)%self.hist_iters==0: 
            self._plot_optStateHist(iteration)
        
    
    def on_batch_end(self, iteration, **kwargs):
        if iteration%self.hist_iters: return
        self._plot_paramHist(iteration)
        
    
    def on_epoch_end(self, iteration, **kwargs):
        if self.plot_loss: self._plot_validLoss(iteration,**kwargs)
        self._plot_metrics(iteration,**kwargs)
            
            
    def on_train_end(self,**kwargs): 
        self.writer.close()
        

# test

In [18]:
# 查看你可以指定哪些metrics，hyper，和parameter
# 先快速地进行一次训练，learner才能有recorder
learn = get_learn(opt=nb_optimizer.SU_Adam)
learn.fit(epochs=1)

print('metrics names:')
for mt in learn.recorder.metrics_names:
    print(f'\t\t{mt}')

print('hyper names:')
for hp in learn.opt.opt_keys:
    print(f'\t\t{hp}')

m = learn.model; 
m = m.module if isinstance(m,torch.nn.DataParallel) else m;
pns = [name for (name, _) in m.named_parameters()]
print('param names:')
for pn in pns:
    print('\t\t{}'.format(pn))

epoch,train_loss,valid_loss,clas_acc,cent_d,hw_r,time
0,6.640443,5.043548,0.221999,0.007879,1.231237,00:46


metrics names:
		clas_acc
		cent_d
		hw_r
hyper names:
		lr
		betas
		weight_decay
param names:
		conv1.weight
		bn1.weight
		bn1.bias
		res_blocks.0.0.conv1.weight
		res_blocks.0.0.bn1.weight
		res_blocks.0.0.bn1.bias
		res_blocks.0.0.conv2.weight
		res_blocks.0.0.bn2.weight
		res_blocks.0.0.bn2.bias
		res_blocks.0.1.conv1.weight
		res_blocks.0.1.bn1.weight
		res_blocks.0.1.bn1.bias
		res_blocks.0.1.conv2.weight
		res_blocks.0.1.bn2.weight
		res_blocks.0.1.bn2.bias
		res_blocks.1.0.conv1.weight
		res_blocks.1.0.bn1.weight
		res_blocks.1.0.bn1.bias
		res_blocks.1.0.conv2.weight
		res_blocks.1.0.bn2.weight
		res_blocks.1.0.bn2.bias
		res_blocks.1.0.downsample.0.weight
		res_blocks.1.0.downsample.1.weight
		res_blocks.1.0.downsample.1.bias
		res_blocks.1.1.conv1.weight
		res_blocks.1.1.bn1.weight
		res_blocks.1.1.bn1.bias
		res_blocks.1.1.conv2.weight
		res_blocks.1.1.bn2.weight
		res_blocks.1.1.bn2.bias
		res_blocks.2.0.conv1.weight
		res_blocks.2.0.bn1.weight
		res_blocks.2.0.bn1.bias


在下面cell的代码开始运行后，去命令行启动（或重新启动）tensorboard，就可以在浏览器上看到输出了。

In [ ]:
# 进行3次训练
# 指定tensorboard加载的总目录
log_root = './tb_log/'
# 先清空加载目录
if os.path.exists(log_root): shutil.rmtree(log_root)
os.mkdir(log_root)
# 进行3次训练
for i in range(3):
    log_dir = f'{log_root}run#{i}'
    plot_net = True if i==0 else False # 仅绘制一次网络结构图
    
    learn = get_learn(opt=nb_optimizer.Adam) # 每次从一个初始化网络开始训练
    tbCb = TensorBoardCallback(learn=learn,
                               log_dir=log_dir,
                               plot_net=plot_net,
                               plot_loss=True,
                               metric_plots=['cent_d'],
                               hyper_plots=['lr'],
                               paramHist_plots=['pred_blocks.1.oconv_clas.weight'],
                               gradHist_plots=['pred_blocks.1.oconv_clas.weight'],
                               optStateHist_plots=['pred_blocks.1.oconv_clas.weight'],
                               hist_iters=5)
    learn.fit(epochs=3,callbacks=[tbCb])

# export

In [1]:
!python notebook2script.py --fname 'tensorboard_callback.ipynb' --outputDir './exp/'

Converted tensorboard_callback.ipynb to exp/nb_tensorboard_callback.py
